In [ ]:
import os 
os.chdir("..")

In [ ]:
import polars as pl
import geopandas as gpd
import pandas as pd
from src.data_pull import DataPull
import requests

dp = DataPull(database_file="data2.ddb")

In [ ]:
df = dp.pull_min_wage()
df.select(pl.col("year")).max()

In [ ]:
var = "area_fips,year,qtr,industry_code,agglvl_code,month1_emplvl,month2_emplvl,month3_emplvl,total_qtrly_wages,avg_wkly_wage,qtrly_estabs"
df = dp.conn.sql(
    f"""
    SELECT {var} FROM 'QCEWTable' 
        WHERE agglvl_code=74;
    """
).pl()
df.filter(pl.col("avg_wkly_wage") == pl.col("avg_wkly_wage").max())

In [ ]:
pl.read_csv("out.csv")

In [ ]:
dp.conn.sql("SELECT * FROM 'QCEWTable';").write_csv("out.csv")

In [ ]:
temp = dp.conn.sql("SELECT DISTINCT area_fips FROM 'QCEWTable' WHERE year=2018 AND qtr=1;").df()
temp["area_fips"] = temp["area_fips"].str.zfill(5)
temp = temp["area_fips"].to_list()

In [ ]:
gdf = gpd.read_file("data/raw/tl_2024_us_county.zip")
gdf["county_id"] = gdf["STATEFP"] + gdf["COUNTYFP"]
remove_list_sates = ["66", "69", "60", "09", "15", "69", "02"]
remove_list_counties = ["46102"]
gdf = gdf[~gdf["STATEFP"].isin(remove_list_sates)]
gdf = gdf[~gdf["county_id"].isin(remove_list_counties)]
county_list = list(gdf["county_id"].values)

In [ ]:
gdf[gdf["county_id"] == "47073"]

In [ ]:
missing = list(set(county_list) - set(temp))
len(missing)

In [ ]:
print(len(county_list))
missing

In [ ]:
dp.conn.sql("SELECT * FROM 'QCEWTable' WHERE year=2018 AND qtr=1 AND area_fips=04007 LIMIT(1);").df()

In [ ]:
int(dp.conn.sql("SELECT MAX(qtr) FROM 'QCEWTable' WHERE year=2019 LIMIT(1);").df().values[0][0]) 

In [ ]:
pl.read_csv("data/raw/bls_2014_1_51685.csv", ignore_errors=True)

In [ ]:
for file in os.listdir("data/raw/"):
    if not file.endswith(".csv"):
        continue
    print(file)
    pl.read_csv(f"data/raw/{file}")

In [ ]:
gdf = gpd.read_file("data/raw/tl_2024_us_county.zip")
gdf["county_id"] = gdf["STATEFP"] + gdf["COUNTYFP"]
# remove_list = ["66010"]
# gdf = gdf[~gdf["county_id"].isin(remove_list)]

In [ ]:

gdf[gdf["county_id"] == "04007"]

In [ ]:
gdf[gdf["STATEFP"] == "02"].plot()

In [ ]:


dp.pull_qcew()

In [ ]:
# os.remove("data/raw/bls_2014_1_66010.csv")
dp.pull_qcew_file(year=2014,qtr=1,county="66010")

In [ ]:
df = pl.read_csv("data/external/county.csv")
df = df.filter(~pl.col("area_title").str.contains("Statewide"))
df = df.filter(~(pl.col("area_fips").str.contains("US")) & ~(pl.col("area_fips").str.contains("C")))
df.filter(pl.col("area_fips")  == "04007")

In [ ]:
df = pl.read_csv("test.csv")
# df = df.filter(pl.col("agglvl_code") == 74)
df = df.select(pl.col(["area_fips", "year", "qtr", "own_code", "industry_code", "month1_emplvl","month2_emplvl", "month3_emplvl", "total_qtrly_wages", "qtrly_estabs"]))
df

In [ ]:
# Parte de web scraping
url = 'https://www.dol.gov/agencies/whd/state/minimum-wage/history'
html = requests.get(url).content
# Recoleccion de datos de cpi paa 2020-2023
df_list = pd.read_html(html, match='2023')
df_2022 = df_list[-1]
# df_2022.drop(columns=df_2022.columns[0], axis=1, inplace=True)

# # Repiticion de web scraping para los demas años
# # Recoleccion de datos de cpi paa 2010-2019
df_list = pd.read_html(html, match='2014')
df_2019 = df_list[-1]
# df_2019.drop(columns=df_2019.columns[0], axis=1, inplace=True)
df = pd.merge(df_2022, df_2019, how='left', on='State or other jurisdiction')
df = pl.from_pandas(df)
df = df.rename({"State or other jurisdiction":"state_name"})
df.unpivot(index="state_name",variable_name="year", value_name="min_wage")